# Custom Code   KNN  Implementation

### Below steps are implemented following pseudo code for knn.


In [42]:
# does float division in py -2 

from __future__ import division

# module to read arff format files 

import arff

In [43]:
training_file="trainProdSelection.arff"

testing_file="testProdSelection.arff"

## Reading arff files into list 

Below function reads content from arff files into py -2 list.


In [68]:
Dtrain = []

Dtest = []

def read_file(arff_file,file_):
    
    for row in arff.load(arff_file):
        file_.append(
                
            [ 
                row['Type'],
                row['LifeStyle'],
                map(float,[row['Vacation'],row['eCredit'],row['salary'],row['property']]),
                row['label']
            ]
            
        )
        
    return file_

Dtrain=read_file(training_file,Dtrain)

Dtest=read_file(testing_file,Dtest)

    

## Data Normalisation

#### normalise_data func is implemented to normalise real values in Dtrain  and Dtest array 

In [69]:
def normalise_data(vector):
    vacation = []
    credit = []
    salary = []
    property_ = []
    
    for y in vector:
        a,b,c,d=y[2]
        vacation.append(a)
        credit.append(b)
        salary.append(c)
        property_.append(d)
    
    normalise = lambda vec,min_,max_ : [(f-min_)/(max_-min_) for f in vec]
    
    vacation=normalise(vacation,min(vacation),max(vacation))
    
    credit = normalise(credit,min(credit),max(credit))
    
    salary = normalise(salary,min(salary),max(salary))
    
    property_ = normalise(property_,min(property_),max(property_))
    
    index = 0
    
    for l in vector:
        l[2]=[vacation[index],credit[index],salary[index],property_[index]]
        index+=1
        
    return vector

#### Normalisation of vector is done 

##### And label column from Dtest is removed  and kept seperate for future validation

In [70]:
Dtrain = normalise_data(Dtrain)

Dtest = normalise_data(Dtest)

verification = lambda Dtest: [row_[3] for row_ in Dtest ]

validation=verification(Dtest)

### KNN Function

#### I observed from similarity matrix that no two attributes are similar

#### and coded that into similar lambda func

#### "similar":   gives 1 if two attr are similar else 0 

#####  "diff" : calulates differences of two vectors 

##### "distance" : gives us euclidean distance 

#####  "top_five": returns least five values from sorted list



In [71]:
def KNN(Dtest,Dtrain):
    
    result = []

    import math
    
    similar =  lambda x,y: 1 if x==y else 0
    
    distance = lambda v1: math.sqrt(sum([v**2 for v in v1]))
    
    diff = lambda v0,v2,l : [v0[n]-v2[n] for n in xrange(l)]
        
    top_five = lambda vec : [vec[i][1] for i in xrange(0,5)]
    
    reverse=lambda dict_1,v:{
        v:sorted([key for key in dict_1 if dict_1[key]==v])
    }
        
    from operator import itemgetter
    
    app_ = result.append
    
    for vec in Dtest:
        distances = []
        
        app=distances.append 
        
        for vec_ in Dtrain:
            a=1-similar(vec[0],vec_[0])
            b=1-similar(vec[1],vec_[1])
            c,d,e,f = diff(vec[2],vec_[2],len(vec_[2]))
            app([distance([a,b,c,d,e,f]),vec_[3]])
        
        distances=sorted(distances,key=itemgetter(0))
        
        classes = top_five(distances)
        
        class_ = {
            'C1': classes.count('C1'),
            'C2': classes.count('C2'),
            'C3': classes.count('C3'),
            'C4': classes.count('C4'),
            'C5': classes.count('C5')
        }
        
        v=max(class_.values())
        
        d=reverse(class_,v)
        
        app_(d[v][0])
        
    return result
                
result=KNN(Dtest,Dtrain)
        

#### Accuracy Function

#### I implemented my own accuarcy function.

#### It calculates total number of matches in validation and result and then divides by total observations.

#### current accuracy is  28.5714

In [84]:
def accuracy(result,validation):
    r1 = len(result)
    
    matrix_2 = {'C1':1,'C2':2,'C3':3,'C4':4,'C5':5}
    
    result = [matrix_2[g] for g in result] if type(result[0])==str else  result
    
    validation = [matrix_2[b] for b in validation] if type(validation[0])==str else validation
        
    matches = lambda v1,v2,r1: [1 if v1[r]==v2[r] else 0 for r in xrange(r1)]    
    
    return matches(result,validation,r1).count(1)/len(result)*100

print "custom KNN implementation accuracy :"+" %f " % accuracy(result,validation)

custom KNN implementation accuracy : 28.571429 


# Sci-Kit Learn Implementation

#### To be implemented for comparison


In [73]:
from sklearn.neighbors import KNeighborsClassifier


In [74]:
KNeighborsClassifier

sklearn.neighbors.classification.KNeighborsClassifier

In [75]:
temp_new = []
Y = []

matrix_0 = {'student':1,'engineer':2,'librarian':3,'professor':4,'doctor':5}

matrix_1 = {'spend>saving':1,'spend>>saving':2,'spend<<saving':3,'spend<saving':4}

matrix_2 = {'C1':1,'C2':2,'C3':3,'C4':4,'C5':5}

for i in  Dtrain:
    a = matrix_0[i[0]]
    b = matrix_1[i[1]]
    c,d,e,f=i[2]
    g = i[3]
    Y.append(matrix_2[g])
    temp_new.append([a,b,c,d,e,f])
test_new = []
for l in Dtest:
    a1 =  matrix_0[l[0]]
    b1 =  matrix_1[l[1]]
    c1,d1,e1,f1 = l[2]
    g1 =  l[3]
    test_new.append([a1,b1,c1,d1,e1,f1])

In [76]:
def distance_function(v1,v2):
    sim= lambda v1,v2,i: 1 if v1[i]==v2[i] else 0
    
    a = sim(v1,v2,0)
    
    
    b = sim(v1,v2,1)
    
    dist = lambda v1,v2,k : sum([(v1[j]-v2[j])**2 for j in xrange(k)])
    
    import math
    
    return math.sqrt(a+b+dist(v1[2:],v2[2:],4))

In [77]:
# from sklearn.neighbors import NearestNeighbors

n=KNeighborsClassifier(
    algorithm='ball_tree',
    n_neighbors=5,
    metric=distance_function,
    weights='distance',
    leaf_size = 60
)

In [78]:
n.fit(temp_new,Y)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=60,
           metric=<function distance_function at 0x000000001C1AE898>,
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')

In [79]:
results = n.predict(test_new)

In [80]:
# validation
# validation = [matrix_2[t] for t in validation]
# print results

In [83]:
print "scikit learn accuracy :"+  " %f " % accuracy(results,validation)


scikit learn accuracy : 19.047619 
